In [1]:
import os
from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification

/Users/saani/.pyenv/versions/3.10.12/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def read_forum_post(text_dir, filename):
    """
    Reads the contents of a forum post from the cadec/text directory.
    Args:
        text_dir (str): Path to the text directory (e.g., 'cadec/text')
        filename (str): Name of the file to read (e.g., 'ARTHROTEC.1.txt')
    Returns:
        str: The contents of the forum post as a string.
    """
    file_path = os.path.join(text_dir, filename)
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()
    return text

In [3]:
def label_text_with_bio(text, model_name="d4data/biomedical-ner-all"):
    """
    Uses a Hugging Face NER pipeline to label each word in the text with BIO/IOB format.
    Args:
        text (str): The input forum post text.
        model_name (str): The Hugging Face model to use.
    Returns:
        list of dict: Each dict contains 'word', 'entity', 'score', 'start', 'end'.
    """
    # Load the model and tokenizer
    ner_pipeline = pipeline("ner", model=model_name, tokenizer=model_name, aggregation_strategy="simple")
    # Run the pipeline on the text
    ner_results = ner_pipeline(text)
    return ner_results

In [4]:
def map_entity_label(entity_label):
    """
    Maps the model's entity label to one of the four required categories: ADR, Drug, Disease, Symptom.
    Returns None if the label should be ignored.
    """
    # Lowercase for robustness
    label = entity_label.lower()
    # Mapping based on typical biomedical NER conventions and CADEC .ann files
    if label in ["medication", "drug","therapeutic_procedure"]:
        return "Drug"
    elif label in ["disease_disorder", "disease"]:
        return "Disease"
    elif label in ["sign_symptom", "symptom"]:
        return "Symptom"
    elif label in ["adr", "adverse_event","sign_symptom", "adverse drug reaction", "adverse reaction","detailed_description"]:
        return "ADR"
    # Add more mappings if needed based on model output
    return None

In [5]:
def convert_ner_to_ann_format(ner_results, label_prefix="T"):
    """
    Converts NER results to the .ann label format used in 'cadec/original',
    mapping model labels to the four required categories.
    Args:
        ner_results (list of dict): Output from the NER pipeline.
        label_prefix (str): Prefix for the tag (default 'T').
    Returns:
        list of str: Each string is a line in the .ann format: Tag<TAB>Label Start End<TAB>Text
    """
    ann_lines = []
    idx = 1
    for entity in ner_results:
        mapped_label = map_entity_label(entity['entity_group'])
        if mapped_label is None:
            continue  # Skip entities not in the required categories
        tag = f"{label_prefix}{idx}"
        start = entity['start']
        end = entity['end']
        text = entity['word']
        ann_line = f"{tag}\t{mapped_label} {start} {end}\t{text}"
        ann_lines.append(ann_line)
        idx += 1
    return ann_lines

In [6]:
def postprocess_ner_results(ner_results, text):
    """
    Postprocesses the NER results to merge subword tokens (e.g., 'dr' + '##owsy' -> 'drowsy'),
    reconstructing full entities and correcting their spans.
    Args:
        ner_results (list of dict): Output from the NER pipeline.
        text (str): The original input text.
    Returns:
        list of dict: Cleaned NER results with merged entities.
    """
    if not ner_results:
        return []
    merged_results = []
    prev = None
    for entity in ner_results:
        # If this is the first entity, just add it
        if prev is None:
            prev = entity.copy()
            continue
        # If the current entity is contiguous with the previous and has the same label, merge them
        if (entity['entity_group'] == prev['entity_group'] and entity['start'] == prev['end']):
            # Merge the words and update the end position
            prev['word'] += entity['word'].replace('##', '')
            prev['end'] = entity['end']
        else:
            # Finalize the previous entity
            # Re-extract the text from the original input to avoid tokenization artifacts
            prev['word'] = text[prev['start']:prev['end']]
            merged_results.append(prev)
            prev = entity.copy()
    # Add the last entity
    if prev is not None:
        prev['word'] = text[prev['start']:prev['end']]
        merged_results.append(prev)
    return merged_results

In [7]:
if __name__ == "__main__":
    # Example: process a single file for demonstration
    example_filename = "ARTHROTEC.1.txt"  # You can change this to any file in cadec/text
    text_dir = os.path.join("cadec", "text")
    # Step 1: Read the forum post
    forum_post = read_forum_post(text_dir, example_filename)
    print("--- Forum Post ---")
    print(forum_post)
    print("------------------")
    # Step 2: Run the model and postprocess the results
    ner_results = label_text_with_bio(forum_post)
    ner_results = postprocess_ner_results(ner_results, forum_post)
    print("\n--- BIO/IOB Labelling Results ---")
    for entity in ner_results:
        print(f"Text: '{entity['word']}' | Label: {entity['entity_group']} | Score: {entity['score']:.2f} | Start: {entity['start']} | End: {entity['end']}")
    print("---------------------------------")
    # Step 3: Convert to .ann format (after postprocessing and mapping)
    ann_lines = convert_ner_to_ann_format(ner_results)
    print("\n--- Converted to .ann Format ---")
    for line in ann_lines:
        print(line)
    # Write predictions to a file for evaluation in step 3
    with open("predicted.ann", "w", encoding="utf-8") as f:
        for line in ann_lines:
            f.write(line + "\n")
    print("\nPredictions written to predicted.ann for evaluation.")

--- Forum Post ---
I feel a bit drowsy & have a little blurred vision, so far no gastric problems.
I've been on Arthrotec 50 for over 10 years on and off, only taking it when I needed it.
Due to my arthritis getting progressively worse, to the point where I am in tears with the agony, gp's started me on 75 twice a day and I have to take it.
every day for the next month to see how I get on, here goes.
So far its been very good, pains almost gone, but I feel a bit weird, didn't have that when on 50.

------------------


Device set to use mps:0



--- BIO/IOB Labelling Results ---
Text: 'drowsy' | Label: Sign_symptom | Score: 1.00 | Start: 13 | End: 19
Text: 'blurred' | Label: Sign_symptom | Score: 0.78 | Start: 36 | End: 43
Text: 'Art' | Label: Detailed_description | Score: 0.97 | Start: 93 | End: 96
Text: 'hrotec' | Label: Therapeutic_procedure | Score: 0.55 | Start: 96 | End: 102
Text: '50' | Label: Dosage | Score: 0.30 | Start: 103 | End: 105
Text: 'over 10 years' | Label: Duration | Score: 1.00 | Start: 110 | End: 123
Text: 'arthritis' | Label: Disease_disorder | Score: 1.00 | Start: 179 | End: 188
Text: '75' | Label: Lab_value | Score: 0.97 | Start: 286 | End: 288
Text: 'twice a day' | Label: Frequency | Score: 0.84 | Start: 289 | End: 300
Text: 'every day' | Label: Frequency | Score: 1.00 | Start: 324 | End: 333
Text: 'for' | Label: Frequency | Score: 0.58 | Start: 334 | End: 337
Text: 'the next month' | Label: Duration | Score: 0.78 | Start: 338 | End: 352
Text: 'pains' | Label: Sign_symptom | Score: 1.00 | Start: 412 |